<a href="https://colab.research.google.com/github/mevangelista-alvarado/crypto_applications/blob/main/CrifradoDeHill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cifrado de Hill

El Cifrado Hill es un cifrado de sustitución basado en el Álgebra Lineal. Fue inventado por Lester S. Hill en 1929.

## Algoritmo

Cada letra está representada por un número. En particular podemos hacer la asignación $A = 0, B = 1, ..., Z = 27$

In [ ]:
import numpy as np
import random
from sympy import Matrix

diccionario_encrypt = {
    'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10,
    'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20,
    'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, '0':26, ' ': 27
}

diccionario_decrypt = {
    '0' : 'A', '1': 'B', '2': 'C', '3': 'D', '4': 'E', '5': 'F', '6': 'G', '7': 'H', '8': 'I', '9': 'J', '10': 'K',
    '11': 'L', '12': 'M', '13': 'N', '14': 'O', '15': 'P', '16': 'Q', '17': 'R', '18': 'S', '19': 'T', '20': 'U',
    '21': 'V', '22': 'W', '23': 'X', '24': 'Y', '25': 'Z', '26': '0', '27': ' '
}

En este método nuestra llave será una matriz de tamaño de $n \times n$

$$
  A_{n\times n} =
  \left[ {\begin{array}{cccc}
    a_{11} & \cdots & a_{1n}\\
    \vdots & \ddots & \vdots\\
    a_{n1} & \cdots & a_{nn}\\
  \end{array} } \right] \pmod{27}
$$
Es decir todos los elementos $a_{ij} \in [0, 1, ..., 26]$

###### Encriptar

Entonces la idea es utilizar
$$A_{n\times n} \cdot x_{n\times 1} = y_{n\times 1} \pmod{27}  $$

donde $A$ es la llave, $x$ es el mensaje e $y$ es el mensaje encriptado.

Si el tamaño del mensaje es mayor a $n$ se codifica en bloques de longitud $n$

###### Ejemplo
Consideramos en el mensaje
${\displaystyle {\begin{pmatrix}0\\2\\19\end{pmatrix}} \pmod{27}}$
que es equivalente ACT


y consideremos la siguiente matriz como llave
${\displaystyle {\begin{pmatrix}6&24&1\\13&16&10\\20&17&15\end{pmatrix}} \pmod{27}}$

entonces en mensaje cifrado es:

${\displaystyle {\begin{pmatrix}6&24&1\\13&16&10\\20&17&15\end{pmatrix}}{\begin{pmatrix}0\\2\\19\end{pmatrix}} = {\begin{pmatrix}67\\222\\319\end{pmatrix}}} \equiv  {\begin{pmatrix}15\\14\\7\end{pmatrix} \pmod{27}} $

y el vector resultante es equivalnete a POH

##### Desencriptar

Para desencriptar, es la siguiente manera
$$A^{-1}_{n\times n}\cdot y_{n\times 1} = x_{n\times 1} \qquad \pmod{27}$$

donde $A$ es la llave, $x$ es el mensaje e $y$ es el mensaje encriptado.

###### Retomando el ejemplo

La inversa es
${\displaystyle A^{-1} = {\begin{pmatrix}8&5&10\\21&8&21\\21&12&8\end{pmatrix}}\pmod {27}}$

Enconces mutiplicando la matriz inversa $A^{-1}$ por el mensaje encriptado $y$

$${\displaystyle {\begin{pmatrix}8&5&10\\21&8&21\\21&12&8\end{pmatrix}}{\begin{pmatrix}15\\14\\7\end{pmatrix}} = {\begin{pmatrix}260\\574\\539\end{pmatrix}} \equiv {\begin{pmatrix}0\\2\\19\end{pmatrix}}{\pmod {27}}}$$

obtenemos el mensaje original $x$

## Implemenetacion en Python


**Observación:**

Falta agregar una condición para determinar cuando una matriz en módulo 27 tiene un inversa. Por lo tanto, en algunas veces el codigo fallará por la matriz que se usa como llave es aleatoria.

In [ ]:
def hill_key(size):
    matrix = []
    L = []
    # Relleno una lista con tantos valores aleatorios como elementos a rellenar en la matriz determinada por size (size * size)
    for x in range(size * size):
        L.append(random.randrange(28))
    # Se crea la matrix clave con los valores generados, de tamaño size * size
    matrix = np.array(L).reshape(size, size)
    return matrix

In [ ]:
size = 2
_hill_key = hill_key(size)
_hill_key

array([[ 7, 26],
       [25, 15]])

In [ ]:
def hill_cipher(message, key):
    ciphertext = ''
    matrix_mensaje = []
    list_temp = []
    cifrado_final = ''
    ciphertext_temp = ''
    cont = 0
    message = message.upper()
    # Si el tamaño del mensaje es menor o igual al tamaño de la clave
    if len(message) <= len(key):
        # Convertir el tamaño del mensaje al tamaño de la clave, si no es igual, se añaden 'X' hasta que sean iguales los tamaños.
        while len(message) < len(key):
            message = message + 'X'
        # Crear la matriz para el mensaje
        for i in range(0, len(message)):
            matrix_mensaje.append(diccionario_encrypt[message[i]])
        # Se crea la matriz
        matrix_mensaje = np.array(matrix_mensaje)
        # Se multiplica la matriz clave por la de mensaje
        cifrado = np.matmul(key, matrix_mensaje)
        # Se obtiene el modulo sobre el diccionario de cada celda
        cifrado = cifrado % 28
        # Se codifica de valores numericos a los del diccionario, añadiendo a ciphertext el valor en el diccionario pasandole como indice la i posicion de la variable cifrado
        for i in range(0, len(cifrado)):
            ciphertext += diccionario_decrypt[str(cifrado[i])]
    else:
    # Si el tamaño del mensaje es menor o igual al tamaño de la clave
        # Si al dividir en trozos del tamaño de la clave, existe algun trozo que tiene menos caracteres que la long. de la clave se añaden tantas 'X' como falten
        while len(message) % len(key) != 0:
            message = message + 'X'
        # Se troce el mensaje en subsstrings de tamaño len(key) y se alamcenan como valores de un array
        matrix_mensaje = [message[i:i + len(key)] for i in range(0,
                          len(message), len(key))]
        # Para cada valor del array (grupo de caracteres de la longitud de la clave)
        for bloque in matrix_mensaje:
            # Crear la matriz para el bloque
            for i in range(0, len(bloque)):
                list_temp.append(diccionario_encrypt[bloque[i]])
            # Se crea la matriz de ese bloque
            matrix_encrypt = np.array(list_temp)
            # Se multiplica la matriz clave por la del bloque
            cifrado = np.matmul(key, matrix_encrypt)
            # Se obtiene el modulo sobre el diccionario de cada celda
            cifrado = cifrado % 28
            # Se codifica de valores numericos a los del diccionario, añadiendo a ciphertext el valor en el diccionario pasandole como indice la i posicion de la variable cifrado
            for i in range(0, len(cifrado)):
                ciphertext_temp += diccionario_decrypt[str(cifrado[i])]
            # Se inicializan las variables para el nuevo bloque
            matrix_encrypt = []
            list_temp = []
        # Se añade el mensaje encriptado a la variable que contiene el mensaje encriptado completo
        ciphertext = ciphertext_temp
    return ciphertext

In [ ]:
mensaje = "HOLA MUNDO"
mesaje_encriptado_hill = hill_cipher(mensaje, _hill_key)
mesaje_encriptado_hill

'VVVXZPCXVF'

In [ ]:
def hill_decipher(message, key):
    plaintext = ''
    matrix_mensaje = []
    plaintext_temp = ''
    list_temp = []
    matrix_inversa = []
    matrix_mensaje = [message[i:i + len(key)] for i in range(0,
                      len(message), len(key))]
    # Se calcula la matriz inversa aplicando el modulo 41
    matrix_inversa = Matrix(key).inv_mod(28)
    # Se transforma en una matriz
    matrix_inversa = np.array(matrix_inversa)
    # Se pasan los elementos a float
    matrix_inversa = matrix_inversa.astype(float)
    # Para cada bloque
    for bloque in matrix_mensaje:
        # Se encripta el mensaje encriptado
        for i in range(0, len(bloque)):
            list_temp.append(diccionario_encrypt[bloque[i]])
        # Se convierte a matriz
        matrix_encrypt = np.array(list_temp)
        # Se multiplica la matriz inversa por el bloque
        cifrado = np.matmul(matrix_inversa, matrix_encrypt)
        # Se le aplica a cada elemento el modulo indicado
        cifrado = np.remainder(cifrado, 28).flatten()
        # Se desencripta el mensaje
        for i in range(0, len(cifrado)):
            plaintext_temp += diccionario_decrypt[str(int(cifrado[i]))]
        matrix_encrypt = []
        list_temp = []
    plaintext = plaintext_temp
    # Se eleminan las X procedentes de su addicion en la encriptacion para tener bloques del tamaño de la clave
    while plaintext[-1] == 'X':
        plaintext = plaintext.rstrip(plaintext[-1])
    return plaintext

In [ ]:
mesaje_desencriptado_hill = hill_decipher(mesaje_encriptado_hill, _hill_key)
mesaje_desencriptado_hill

'HOLA MUNDO'